In [138]:

##### IMPORTS #####
import pandas as pd

from sklearn.model_selection import train_test_split

import numpy as np

import torch as T
import torch.nn as nn
import torch.optim as optim

from torchvision.io import read_image
from torchvision import transforms


##### FUNCTION #####
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv3d(3,10, kernel_size=16, stride = 2),
            nn.ReLU(inplace=True),

            #nn.BatchNorm1d(16),
            #nn.Dropout(0.2),
            
            #nn.Conv2d(16,10 , kernel_size=8, stride = 2),
            #nn.ReLU(inplace=True),
            
            #nn.BatchNorm1d(8),
            #nn.Dropout(0.2),
            
            #nn.Conv2d(8, 10, kernel_size=4, stride = 2),
            #nn.ReLU(inplace=True),
            
            #nn.BatchNorm1d(4),
            #nn.Dropout(0.2),
            
            nn.Flatten()
            
        )
        
        self.classifier = nn.Sequential(
            #nn.Linear(4, 32),
            #nn.Linear(8, 16),
            #nn.Linear(16, 8), derniere sortie nombre de classes
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Import data
def image_to_tensor(path) :
    try : 
        return read_image(path)
    
    except :
        return None

def import_and_clean(path) :
    # Import csv
    df = pd.read_csv(path+"labels.csv",sep =",").drop("Unnamed: 0", axis=1)
    df["len"] = 5-df["Image"].astype(str).str.len()
    df["zeroes"] =  "0"
    df["Image"] =  "BloodImage_" + df["zeroes"] * df["len"] +df["Image"].astype(str) + ".jpg"
    
    
    #cleanup csv
    df = df.drop(["len", "zeroes"], axis=1)
    df = df[df['Category'].notnull()]

    #Import images
    df["Tensor"] = df["Image"].apply(lambda x : image_to_tensor(path+"JPEGImages/"+x))
    
    df = df[df["Tensor"].notnull()]
    
    return df["Category"], T.stack(list(df["Tensor"]))


##### MAIN ######
path = "data/dataset-master/"
transform = transforms.Compose([transforms.Resize((224,320)), transforms.ToTensor()])

labels,features = import_and_clean(path)

train_features, validation_features, train_labels, validation_labels = train_test_split(features, labels, test_size = 0.05)
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels, test_size = 0.2)



In [155]:
from torch.nn import Module
class LeNet(Module):
    def __init__(self, numChannels, classes):
        # call the parent constructor
        super(LeNet, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = nn.Conv2d(in_channels=numChannels, out_channels=20,
            kernel_size=(5, 5))
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50,
            kernel_size=(5, 5))
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = nn.Linear(in_features=800, out_features=500)
        self.relu3 = nn.ReLU()
        # initialize our softmax classifier
        self.fc2 = nn.Linear(in_features=500, out_features=classes)
        self.logSoftmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        # flatten the output from the previous layer and pass it
        # through our only set of FC => RELU layers
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output


In [156]:
net = LeNet(3,2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=1e-3)
batch_size=1

for t in range(2):
    optimizer.zero_grad()
    predictions = net(train_features)
    loss = criterion(predictions,train_labels)
    loss.backward()
    optimizer.step()

RuntimeError: expected scalar type Byte but found Float